![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FServing&file=Understanding+Prediction+IO+With+FastAPI.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Understanding%20Prediction%20IO%20With%20FastAPI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FServing%2FUnderstanding%2520Prediction%2520IO%2520With%2520FastAPI.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Understanding%20Prediction%20IO%20With%20FastAPI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Serving/Understanding%20Prediction%20IO%20With%20FastAPI.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Understanding Prediction IO With FastAPI

**IN PROGRESS**

Use a simple FastAPI implementation to explore the processing of different input instance format on:
- Vertex AI Endpoints
- Cloud Run Endpoints
- Vetex AI Batch Prediction

Inputs to try:
- https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#input_data_requirements
- https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions#formatting-prediction-input


---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('docker', 'docker'),
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.artifactregistry_v1', 'google-cloud-artifact-registry'),
    ('google.cloud.devtools', 'google-cloud-build'),
    ('google.cloud.run_v2', 'google-cloud-run'),   
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [2]:
!gcloud services enable artifactregistry.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable run.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [3]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [5]:
REGION = 'us-central1'
SERIES = 'mlops-serving'
EXPERIMENT = 'understand-io'

GCS_BUCKET = PROJECT_ID

packages:

In [6]:
import json, os
import time
import requests

import docker

import google.auth
from google.cloud import storage
from google.cloud import artifactregistry_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import run_v2
from google.cloud import aiplatform

clients:

In [7]:
# gcs storage client
gcs = storage.Client(project = GCS_BUCKET)
bucket = gcs.bucket(GCS_BUCKET)

# cloud build client
cb = cloudbuild_v1.CloudBuildClient()

# artifact registry client
ar = artifactregistry_v1.ArtifactRegistryClient()

# cloud run client
cr = run_v2.ServicesClient()

# vertex ai client
aiplatform.init(project = PROJECT_ID, location = REGION)

Parameters:

In [8]:
DIR = f"files/{EXPERIMENT}"

Environment:

In [9]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Build A Custom Prediction Container

It is really not all that hard with Python!

For this example [FastAPI](https://fastapi.tiangolo.com/) is used.

This process uses docker to build a custom container and then runs the container on Cloud Run and Vertex AI Endpoints.

This could be done locally with Docker and pushed to Artifact Registry before deployment to Cloud Run.  The process below assumes that docker is not available locally and used Cloud Build to both build and push the resulting container to Artifact Registry.

---
### Setup Artifact Registry

[Artifact registry](https://cloud.google.com/artifact-registry/docs) organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).

#### List Repositories

This may be empty if no repositories have been created for this project

In [10]:
for repo in ar.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks
projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks-catboost
projects/statmike-mlops-349915/locations/us-central1/repositories/gcf-artifacts
projects/statmike-mlops-349915/locations/us-central1/repositories/mlops
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


#### Create/Retrieve Docker Image Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create one named for this project.

In [11]:
docker_repo = None
for repo in ar.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{SERIES}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{SERIES}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {SERIES} series that holds docker images.',
                name = f'{SERIES}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Creating Repository ...
Completed creating repo: projects/statmike-mlops-349915/locations/us-central1/repositories/mlops-serving


In [12]:
docker_repo.name, docker_repo.format_.name

('projects/statmike-mlops-349915/locations/us-central1/repositories/mlops-serving',
 'DOCKER')

In [13]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [14]:
REPOSITORY

'us-central1-docker.pkg.dev/statmike-mlops-349915/mlops-serving'

---
### Create Application Files

```
|__ Dockerfile
|__ requirements.txt
|__ app
    |__ __init__.py
    |__ main.py
    |__ prestart.sh
```

In [15]:
if not os.path.exists(DIR + '/source/app'):
    os.makedirs(DIR + '/source/app')

In [16]:
%%writefile {DIR}/source/Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY ./requirements.txt requirements.txt

RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir -r requirements.txt

Writing files/understand-io/source/Dockerfile


In [17]:
%%writefile {DIR}/source/requirements.txt
numpy

Writing files/understand-io/source/requirements.txt


In [18]:
%%writefile {DIR}/source/app/__init__.py
# init file

Writing files/understand-io/source/app/__init__.py


In [19]:
%%writefile {DIR}/source/app/main.py
# a simple passthrough of instance to predictions

# packages
import os
from fastapi import FastAPI, Request
import numpy as np

# clients
app = FastAPI()

# Define function for health route
@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

# Define function for prediction route
@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    # await the request
    body = await request.json()
    
    # parse the request
    instances = body["instances"]
    
    # return the received inputs as the "predictions" - a simple pass through
    predictions = instances

    # this returns just the predicted probabilities:
    return {"predictions": predictions}

Writing files/understand-io/source/app/main.py


In [20]:
%%writefile {DIR}/source/app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing files/understand-io/source/app/prestart.sh


In [21]:
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/Dockerfile').upload_from_filename(f'{DIR}/source/Dockerfile')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/requirements.txt').upload_from_filename(f'{DIR}/source/requirements.txt')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/__init__.py').upload_from_filename(f'{DIR}/source/app/__init__.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/main.py').upload_from_filename(f'{DIR}/source/app/main.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/prestart.sh').upload_from_filename(f'{DIR}/source/app/prestart.sh')

In [22]:
list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/source'))

[<Blob: statmike-mlops-349915, mlops-serving/understand-io/source/Dockerfile, 1740159897721281>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/app/__init__.py, 1740159897918644>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/app/main.py, 1740159897981494>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/app/prestart.sh, 1740159898044525>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/requirements.txt, 1740159897860198>]

---
### Build Application Container

Use the Cloud Build client to construct and run the build instructions. Here the files collected in GCS are copied to the build instance, then the Docker build is run in the folder with the `Dockerfile`. The resulting image is pushed to Artifact Registry (setup above).

In [23]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/source/*', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}"]

In [24]:
operation = cb.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [25]:
build_response = operation.result()
build_response.status, build_response.artifacts

(<Status.SUCCESS: 3>,
 images: "us-central1-docker.pkg.dev/statmike-mlops-349915/mlops-serving/understand-io")

In [26]:
build_response.artifacts.images[0]

'us-central1-docker.pkg.dev/statmike-mlops-349915/mlops-serving/understand-io'

---
## Test Locally

If Docker is installed and running locally then use it to test the image.

In [27]:
try:
    local_test = True
    docker_client = docker.from_env()
    if docker_client.ping():
        print(f"Docker is installed and running. Version: {docker_client.version()['Version']}")
except Exception as e:
    local_test = False
    print('Docker is either not installed or not running - please fix before proceeding.\nhttps://docs.docker.com/engine/install/')

Docker is installed and running. Version: 20.10.17


### Pull and Run Container

Run the container image with:
- ports: inside 8080 mapped to outside 80
- set environment variables for:
    - `AIP_HTTP_PORT` is `8080`
    - `AIP_HEALTH_ROUTE` is `/health`
    - `AIP_PREDICT_ROUTE` is `/predict`
    - `AIP_STORAGE_URI` is the `gs://bucket/path/to/folder`
    - `MODULE_NAME` is 'main'
        - this actually defaults to main so is not required
        - an alternative script with different prediction output is created in `main2.py` above
        - use this environment variable to start the container using the alternative script in module `main2`
        - see the [FastAPI Docker Image Advanced Usage](https://github.com/tiangolo/uvicorn-gunicorn-fastapi-docker?tab=readme-ov-file#advanced-usage) details

In [43]:
if local_test:
    # make sure any prior runs are stopped:
    try:
        container = docker_client.containers.get('local-run')
        container.stop()
        container.remove()
    except docker.errors.NotFound:
        pass
    
    # get image:
    image_uri = build_response.artifacts.images[0]
    try:
        local_image = docker_client.images.get(image_uri)
        remote_image = docker_client.images.pull(image_uri)
        if local_image.id != remote_image.id:
            print('New image found, updating ...')
            local_image = remote_image
        else:
            print('Using existing image ...')
    except docker.errors.ImageNotFound:
        print('Pulling image ...')
        local_image = docker_client.images.pull(image_uri)
        
    # run container:
    print('Starting container ...')
    container = docker_client.containers.run(
        image = image_uri,
        detach = True,
        ports = {'8080/tcp':80}, # Map inside:outside (where docker run -p is outside:inside)
        name = 'local-run',
        environment = {
            'AIP_HTTP_PORT': '8080',
            'AIP_HEALTH_ROUTE': '/health',
            'AIP_PREDICT_ROUTE': '/predict',
            'AIP_STORAGE_URI': f'gs://{bucket.name}/{SERIES}/catboost-overview',
            'MODULE_NAME': 'main' # try main2 for alternative output
        }
    )
    print('Container ready.\n\tUse `container.logs()` to view startup logs.')

Using existing image ...
Starting container ...
Container ready.
	Use `container.logs()` to view startup logs.


In [47]:
#container.logs()

### Health Check

Want to see `200`:

In [48]:
if local_test:
    response = requests.get(f"http://localhost:80/health")
    print(response.status_code)

200


### Inference Test

In [49]:
def predict(instances):
    url = f"http://localhost:80/predict"
    headers = {'Content_Type': 'application/json'}
    data = json.dumps({'instances': instances})
    response = requests.post(url, headers = headers, data = data)    
    return json.loads(response.text)

In [51]:
predict([1])

{'predictions': [1]}

In [52]:
predict([1,2,3,4])

{'predictions': [1, 2, 3, 4]}

In [54]:
predict([{'key':'value'}])

{'predictions': [{'key': 'value'}]}

In [55]:
predict([{'key':'value'}, {'key':'value2'}])

{'predictions': [{'key': 'value'}, {'key': 'value2'}]}

In [56]:
predict(['gs://path/to/image/image1.jpg'])

{'predictions': ['gs://path/to/image/image1.jpg']}

### Stop Container

In [57]:
container.name

'local-run'

In [58]:
container = docker_client.containers.get(container.name)

In [59]:
container.status

'running'

In [60]:
container.stop()
container.remove()